In [1]:
using LinearAlgebra
using Plots
using GaussQuadrature
using SparseArrays
using DataFrames
using Printf

## Base

In [2]:
function ϕ(a::Int64)::Function
    if a == 1
        return (ξ₁::Float64, ξ₂::Float64) -> (1.0-ξ₁)*(1.0-ξ₂)/4.0
        
    elseif a == 2
        return (ξ₁::Float64, ξ₂::Float64) -> (1.0+ξ₁)*(1.0-ξ₂)/4.0
        
    elseif a == 3
        return (ξ₁::Float64, ξ₂::Float64) -> (1.0+ξ₁)*(1.0+ξ₂)/4.0
        
    elseif a == 4
        return (ξ₁::Float64, ξ₂::Float64) -> (1.0-ξ₁)*(1.0+ξ₂)/4.0
        
    end
end

function ∂ϕ(variable::Int64, a::Int64)::Function
    if variable == 1
        if a == 1
            return (ξ₁::Float64, ξ₂::Float64) -> -(1.0-ξ₂)/4.0
        elseif a == 2
            return (ξ₁::Float64, ξ₂::Float64) ->  (1.0-ξ₂)/4.0
        elseif a == 3
            return (ξ₁::Float64, ξ₂::Float64) ->  (1.0+ξ₂)/4.0
        elseif a == 4
            return (ξ₁::Float64, ξ₂::Float64) -> -(1.0+ξ₂)/4.0
        end
        
    elseif variable == 2
        if a == 1
            return (ξ₁::Float64, ξ₂::Float64) -> -(1.0-ξ₁)/4.0
        elseif a == 2
            return (ξ₁::Float64, ξ₂::Float64) -> -(1.0+ξ₁)/4.0
        elseif a == 3
            return (ξ₁::Float64, ξ₂::Float64) ->  (1.0+ξ₁)/4.0
        elseif a == 4
            return (ξ₁::Float64, ξ₂::Float64) ->  (1.0-ξ₁)/4.0
        end
        
    end
end

∂ϕ (generic function with 1 method)

In [3]:
function mapper_to_x(h::Float64, pᵉ::Float64)::Function
    return (ξ::Float64) -> (h/2)*(ξ+1) + pᵉ
end

mapper_to_x (generic function with 1 method)

In [4]:
function quadratura_phis_K_2D(f1::Function, f2::Function, P::Vector{Float64}, W::Vector{Float64})::Float64
    quadratura = 0.0
    
    for (ξ₁, w₁) in zip(P, W)
        for (ξ₂, w₂) in zip(P, W)
            quadratura += (w₁*w₂) * (f1(ξ₁, ξ₂) * f2(ξ₁, ξ₂))
        end
    end
    
    return quadratura
end

function quadratura_F_2D(f1::Function, f2::Function, h::Vector{Float64}, pᵉ::Vector{Float64}, P::Vector{Float64}, W::Vector{Float64})::Float64
    quadratura = 0.0
    
    ξ_to_x1 = mapper_to_x(h[1], pᵉ[1])
    ξ_to_x2 = mapper_to_x(h[2], pᵉ[2])
    
    for (ξ₁, w₁) in zip(P, W)
        for (ξ₂, w₂) in zip(P, W)            
            quadratura += (w₁*w₂) * f1(ξ_to_x1(ξ₁), ξ_to_x2(ξ₂)) * f2(ξ₁, ξ₂)
        end
    end
    
    return quadratura
endMonta locais

quadratura_F_2D (generic function with 1 method)

In [8]:
function monta_K_local_2D(α::Float64, β::Float64, h::Vector{Float64}, P::Vector{Float64}, W::Vector{Float64})::Matrix{Float64}
    K_local = zeros(4, 4)
    
    for a in 1:4
        for b in 1:4
            K_local[a, b] = (α*h[2]/h[1]  ) * quadratura_phis_K_2D(∂ϕ(1,b), ∂ϕ(1,a), P, W) +
                            (α*h[1]/h[2]  ) * quadratura_phis_K_2D(∂ϕ(2,b), ∂ϕ(2,a), P, W) +
                            (β*h[1]*h[2]/4) * quadratura_phis_K_2D( ϕ(  b),  ϕ(  a), P, W)
        end
    end
    
    return K_local
end

function monta_F_local_2D(α::Float64, β::Float64, f::Function, h::Vector{Float64}, pᵉ::Vector{Float64}, P::Vector{Float64}, W::Vector{Float64})
    F_local = zeros(4)
    
    for a in 1:4
         F_local[a] = (h[1]*h[2]/4) * quadratura_F_2D(f, ϕ(a), h, pᵉ, P, W)
    end
    
    return F_local    
end

monta_F_local_2D (generic function with 1 method)

In [6]:
h = [1/4; 1/4]
α = 6.0
β = 0.0

f = (x1::Float64, x2::Float64) -> 4/(h[1]*h[2])
pᵉ = [0.0; 0.0]
P, W = legendre(2)

display(monta_K_local_2D(α, β, h, P, W))
display(monta_F_local_2D(α, β, f, h, pᵉ, P, W))

4×4 Matrix{Float64}:
  4.0  -1.0  -2.0  -1.0
 -1.0   4.0  -1.0  -2.0
 -2.0  -1.0   4.0  -1.0
 -1.0  -2.0  -1.0   4.0

4-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0

In [11]:
h = [1/4; 1/4]
α = 0.0
β = (9.0 * 4.0)/ (h[1] * h[2])

f = (x1::Float64, x2::Float64) -> (16/(h[1]*h[2])^2)*9*x1*x2
pᵉ = [0.0; 0.0]
P, W = legendre(2)

display(monta_K_local_2D(α, β, h, P, W))
display(monta_F_local_2D(α, β, f, h, pᵉ, P, W))

4×4 Matrix{Float64}:
 4.0  2.0  1.0  2.0
 2.0  4.0  2.0  1.0
 1.0  2.0  4.0  2.0
 2.0  1.0  2.0  4.0

4-element Vector{Float64}:
  4.0
  8.0
 16.0
  8.0